# Laboratorio de regresión - 4

|                |   |
:----------------|---|
| **Nombre**     | Emiliano Valderrama del Toro  |
| **Fecha**      |  06/09/2025 
| **Expediente** |  744673 |

## Modelos penalizados

Hasta ahora la función de costo que usamos para decidir qué tan bueno es nuestro modelo al momento de ajustar es:

$$ \text{RSS} = \sum_{i=1}^n e_i^2 = \sum_{i=1}^n (y_i - \hat{y_i})^2 $$

Dado que los errores obtenidos son una combinación de sesgo y varianza, puede ser que se sesgue un parámetro para minimizar el error. Esto significa que el modelo puede decidir que la salida no sea una combinación de los factores, sino una fuerte predilección sobre uno de los factores solamente. 

E.g. se quiere ajustar un modelo

$$ \hat{z} = \hat{\beta_0} + \hat{\beta_1} x + \hat{\beta_2} y $$

Se ajusta el modelo y se decide que la mejor decisión es $\hat{\beta_1} = 10000$ y $\hat{\beta_2}=50$. Considera limitaciones de problemas reales:
- Quizás los parámetros son ajustes de maquinaria que se deben realizar para conseguir el mejor producto posible, y que $10000$ sea imposible de asignar.
- Quizás los datos actuales están sesgados y sólo hacen parecer que uno de los factores importa más que el otro.

Una de las formas en las que se puede mitigar este problema es penalizando a los parámetros del modelo, cambiando la función de costo:

$$ \text{RSS}_{L2} = \sum_{i=1}^n e_i^2  + \lambda \sum_{j=1}^p \hat{\beta_j}^2 $$

El *L2* significa que se está agregando una penalización de segundo orden. Lo que hace esta penalización es que los factores ahora sólo tendrán permitido crecer si hay una reducción al menos proporcional en el error (sacrificamos sesgo, pero reducimos la varianza).

Asimismo, existe la penalización *L1*

$$ \text{RSS}_{L1} = \sum_{i=1}^n e_i^2  + \lambda \sum_{j=1}^p |\hat{\beta_j}| $$

A las penalizaciones *L2* y *L1* se les conoce también como Ridge y Lasso, respectivamente.

Para realizar una regresión con penalización de Ridge o de Lasso usamos el objeto `Ridge(alpha=?)` o `Lasso(alpha=?)` en lugar de `LinearRegression()` de `sklearn`.

Utiliza el dataset de publicidad (Advertising.csv) y realiza 3 regresiones múltiples:

$$ \text{sales} = \beta_0 + \beta_1 (\text{TV}) + \beta_2 (\text{radio}) + \beta_3 (\text{newspaper}) + \epsilon $$

1. Sin penalización
2. Con penalización L2
3. Con penalización L1

Compara los resultados de los parámetros y sus *p-values*, y los $R^2$ resultantes.

In [68]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from scipy import stats

In [190]:
data = pd.read_csv("Advertising.csv")
data.head()

,Unnamed: 0,TV,radio,newspaper,sales
0,1,230.10000000,37.80000000,69.20000000,22.10000000
1,2,44.50000000,39.30000000,45.10000000,10.40000000
2,3,17.20000000,45.90000000,69.30000000,9.30000000
3,4,151.50000000,41.30000000,58.50000000,18.50000000
4,5,180.80000000,10.80000000,58.40000000,12.90000000


In [191]:
df = data.dropna().drop_duplicates()

In [192]:
x = data[["TV", "radio", "newspaper"]]
y = data[["sales"]]

In [193]:
# Escalar para Ridge y Lasso

X = sm.add_constant(x)

scaler = StandardScaler().fit(X)
x_scaled = scaler.transform(X)

## Sin penalización

In [194]:
lr = LinearRegression()
lr.fit(x, y)

lr.coef_

array([[ 0.04576465,  0.18853002, -0.00103749]])

In [195]:
lr.intercept_

array([2.93888937])

In [196]:
y_pred_lr = lr.predict(x)

In [197]:
lrr2 = r2_score(y, y_pred_lr)

**p_values**

In [198]:
n = x.shape[0]
p = x.shape[1] + 1  

In [199]:
# Hay que juntar el intercepto con las betas siguientes para poder dividir y sacar el t
beta_hat = np.concatenate((lr.intercept_, np.ravel(lr.coef_)))

RSS = np.sum((y_pred_lr - y.to_numpy())**2)
RSE = np.sqrt(( RSS / (n-p)))

var_beta = np.linalg.inv(X.T @ X) * RSE**2
std_beta = np.sqrt(var_beta.diagonal())

t_stats = beta_hat / std_beta
p_values = 2 * (1 - stats.t.cdf(np.abs(t_stats), df=n-p))

In [200]:
p_values

array([0.        , 0.        , 0.        , 0.85991505])

## RIDGE

In [201]:
rd = Ridge(alpha=1)
rd.fit(x_scaled, y)

y_pred_rd = rd.predict(x_scaled)

In [202]:
betas = np.ravel(rd.coef_)

In [203]:
betas[0] = rd.intercept_[0]

In [204]:
r2rd = r2_score(y, y_pred_rd)
r2rd

0.8971891437493419

**p_values_**

In [205]:
RSS = np.sum((y_pred_rd - y.to_numpy())**2)
RSE = np.sqrt(( RSS / (n-p)))

var_beta = np.linalg.inv(X.T @ X) * RSE**2
std_beta = np.sqrt(var_beta.diagonal())

t_stats = betas / std_beta
p_values_rd = 2 * (1 - stats.t.cdf(np.abs(t_stats), df=n-p))

p_values_rd

array([0.        , 0.        , 0.        , 0.00696084])

In [206]:
for alpha in range(1, 11):  # valores de 1 a 10
    rd = Ridge(alpha=alpha)
    rd.fit(x_scaled, y)
    
    y_pred_rd = rd.predict(x_scaled)
    r2 = r2_score(y, y_pred_rd)
    
    print(f"Alpha: {alpha}")
    print(f"  Intercepto: {rd.intercept_}")
    print(f"  Coeficientes: {rd.coef_}")
    print(f"  R²: {r2}")
    print("-" * 40)


Alpha: 1
  Intercepto: [14.0225]
  Coeficientes: [[ 0.          3.90021344  2.77691168 -0.0160149 ]]
  R²: 0.8971891437493419
----------------------------------------
Alpha: 2
  Intercepto: [14.0225]
  Coeficientes: [[ 0.          3.8813594   2.76195013 -0.00963266]]
  R²: 0.8971255841820664
----------------------------------------
Alpha: 3
  Intercepto: [14.0225]
  Coeficientes: [[ 0.00000000e+00  3.86268878e+00  2.74717418e+00 -3.38829519e-03]]
  R²: 0.8970213093745575
----------------------------------------
Alpha: 4
  Intercepto: [14.0225]
  Coeficientes: [[0.00000000e+00 3.84419890e+00 2.73258002e+00 2.72170396e-03]]
  R²: 0.896877623183171
----------------------------------------
Alpha: 5
  Intercepto: [14.0225]
  Coeficientes: [[0.         3.82588713 2.71816398 0.00870072]]
  R²: 0.8966957852701762
----------------------------------------
Alpha: 6
  Intercepto: [14.0225]
  Coeficientes: [[0.         3.80775087 2.70392245 0.01455203]]
  R²: 0.8964770128633048
--------------------

El alpha más funcional para este modelo de Ridge es el de 1, porque noto que entre más lo aumentamos menor se hace el r2, por lo que el modelo se vuelve peor. 

## LASSO

In [207]:
ls = Lasso(alpha=0.1)
ls.fit(x_scaled, y)

y_pred_ls = ls.predict(x_scaled)

In [208]:
ls.coef_

array([0.        , 3.82360787, 2.68932395, 0.        ])

In [209]:
ls.intercept_

array([14.0225])

In [210]:
r2ls = r2_score(y, y_pred_ls)
r2ls

0.896494232900825

In [211]:
betas = np.ravel(ls.coef_)
betas[0] = ls.intercept_[0]

In [212]:
RSS = np.sum((y_pred_ls - y.to_numpy())**2)
RSE = np.sqrt(( RSS / (n-p)))

var_beta = np.linalg.inv(X.T @ X) * RSE**2
std_beta = np.sqrt(var_beta.diagonal())

t_stats = betas / std_beta
p_values_ls = 2 * (1 - stats.t.cdf(np.abs(t_stats), df=n-p))

p_values_ls

array([4.54363523e-01, 0.00000000e+00, 4.81653927e-07, 1.00000000e+00])

In [213]:
pd.set_option("display.float_format", "{:.8f}".format)

In [165]:
resultados = pd.DataFrame({
    "Variable" : ["Intercepto", "TV", "radio", "newspaper"],
    "OLS" : [results.params[0]] + [results.params[1]] + [results.params[2]] + [results.params[3]],
    "Ridge" : [rd.intercept_[0]] + [rd.coef_[0,1]] + [rd.coef_[0,2]] + [rd.coef_[0,3]],
    "Lasso" : [ls.intercept_[0]] + [ls.coef_[0]] + [ls.coef_[1]] + [ls.coef_[2]]
})

resultados

,Variable,OLS,Ridge,Lasso
0,Intercepto,2.93888937,14.02250000,14.02250000
1,TV,0.04576465,3.73691112,14.02250000
2,radio,0.18853002,2.64863338,3.82360787
3,newspaper,-0.00103749,0.03674246,2.68932395


In [221]:
p_values = pd.DataFrame({
    "p_values" : ["Intercepto", "TV", "radio", "newspaper"],
    "LinearRegression" : [p_values[0]] + [p_values[1]] + [p_values[2]] + [p_values[3]],
    "Ridge" : [p_values_rd[0]] + [p_values_rd[1]] + [p_values_rd[2]] + [p_values_rd[3]],
    "Lasso" : [p_values_ls[0]] + [p_values_ls[1]] + [p_values_ls[2]] + [p_values_ls[3]]
})

p_values

,p_values,LinearRegression,Ridge,Lasso
0,Intercepto,0.00000000,0.00000000,0.45436352
1,TV,0.00000000,0.00000000,0.00000000
2,radio,0.00000000,0.00000000,0.00000048
3,newspaper,0.85991505,0.00696084,1.00000000


In [17]:
r2_df = pd.DataFrame({
    "Modelo": ["OLS", "Ridge", "Lasso"],
    "R²": [results.rsquared, r2rd, r2ls]
})
r2_df

,Modelo,R²
0,OLS,0.89721064
1,Ridge,0.89718914
2,Lasso,0.89721042


### Conclusiones

Le voy a ser sincero. En la actividad sí hice uso de chat, pero más que nada para encargarme de entender todo. 
Al principio no entendía el formato en el que debía de utilizar Ridge y Lasso. Luego me di cuenta que es igual que LinearRegression lo cual lo hizo más simple. Sin embargo, veo que en estos dos modelos es importante escalar los datos para que la penaización no afecte demasiado a los coeficientes. Y bueno ya lo demás era igual entonces ahí lo comprendí bien. Y en la parte de interpretación de los resultados:

En el modelo OLS los p_values nos indican que las variables TV y radio son significativamente importantes para el modelo, pero newspaper tiene un p_value muy alto, lo que indica que no llega a ser relevante para las sales. Justo algo que ya habíamos visto en clases pasadas, y que al hacer regresión por separada ahí si parecería ser importante, cosa que en la regresión múltiple cambió.

Los resultados de Ridge y Lasso son meramente parecidos, tanto en coeficientes como intercepto. Aunque el r2 sale más alto en Lasso, quedando en segundo lugar después de OSL.

Algo que me llamó la atención fue que el intercepto en Ridge y Lasso son muy altos, casi 7 veces más alto que en OSL. Esto porque en la penalización el intercepto no se afecta. Los coeficiente se achican y ese aumenta para controlar la alteración.

Y bueno, en todos los modelos nos damos cuenta de que el periódico no es muy relevante para determinar las ventas, y no solo eso, sino que afecta de forma negativa. Pienso que esto puede ser a razón de que ya es un medio publicitario que día con día se utiliza menos, por lo que es un gasto que vale la pena removerlo. 

Y para terminar, vemos que OLS es el modelo que mejor se ajusta gracias al R2, aunque bueno, la diferencia es muy poca. Además, en el modelo OLS la radio llega a ser más importante que la TV y en los otros dos modelos es lo contrario.
Yo me iría por el modelo Lasso. Me parece el más eficiente y congruente, ya que yo consideraría que es más importante la TV que la radio también. 